In [89]:
import os
import pandas as pd
import numpy as np
import datetime
state_dict = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ',
    'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 
    'Connecticut': 'CT', 'District Of Columbia':"DC", 'Delaware': 'DE', 'Florida': 'FL', 
    'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA','Kansas': 'KS', 'Kentucky': 'KY',
    'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI',
    'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX',
    'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', "United States": "US", "United States Total": "US"}


In [90]:
def is_substring_in_array(substring, array):
    # Use np.char.find to find the substring in each element of the array
    found = np.char.find(array, substring)
    # Check if any position is not -1
    return np.any(found != -1)

In [91]:

dfs_dct = {}
ab_keys = {"a": "United States", "b": "Missouri"}
start_year = 1998
end_year = 2022
folder = "StateFinancesByYear"
null_years = [1997, 2001, 2003, 2004]
for year in range(start_year,end_year):
    if year in null_years:
        continue
    dfs_dct[year] = {}        
    stryear = str(year)
    filetype = ".xlsx" if year > 2011 else ".xls"

    for ab in ["a", "b"]:
        filename = f"{folder}/StateGovFinances{stryear}{ab}{filetype}"
        try:
            df = pd.read_excel(filename)#dropna(axis = 0, how = "any")
        except:
            print(stryear+ab, "failed")
            break
        index = [key for key in df if "Table" in key]
        if len(index) != 0:
            df = df.set_index(index)#.iloc[1:]
        else:
            df = df.set_index(df.columns[0])
        df.index = ([str(ind).split('(')[0].strip() for ind in df.index])

        if year  < 1998:
            df[list(df.keys())[:]]
            df = df[list(df.keys())[4:]]

        for row in range(df.shape[0]):
            in_array = is_substring_in_array(ab_keys[ab].title(), str(df.iloc[row]).title())
            if in_array:
                break #if is_string_array(df.iloc[row].values) else True:
            else:
                continue

        df.iloc[row] = df.iloc[row].str.title() 
        df.iloc[row] = df.iloc[row].fillna(method='ffill')
        df.iloc[row + 1] = df.iloc[row + 1].str.strip() + " " + df.iloc[row +2].str.replace("1", "").str.strip()+ " " + df.iloc[row +3].str.replace("1", "").str.strip()
        index = list(df.index)
        index[row+1] = "Variable"
        # df.set_index(index, inplace = True)
        # df.set_index(list(df.keys())[row, row + 1]) 
        # df.columns = df.iloc[[row, row + 1]].T

        # df = df.iloc[row + 6:].dropna(how = "all")
        df = df.iloc[row:].dropna(how = "all")

        df.index = [str(ix).strip().replace("1", "").upper() for ix in df.index]
        df_index_count = df.index.to_series().groupby(df.index).cumcount().add(1).astype(str)
        df.index = df_index_count + df_index_count.index
        tdf = df.dropna(how = "all").T
        cols = list(tdf.iloc[1:].columns)
        cols[0] = "State"
        cols[1] = "Format"
        tdf.columns = cols
        tdf = tdf.iloc[1:]
        df = tdf    
        df["State"] = df["State"].apply(lambda x: state_dict[x.strip()])     
        df["Year"] = datetime.datetime(year,1,1)
        df.set_index(["State", "Year", "Format"], inplace = True)
        keys = list(df.keys())[4:]
        
        dfs_dct[year][ab] = df[keys]#reset_index().set_index(["Year", "index"])#.T
for year, ab_dct in dfs_dct.items():
    if "b" in ab_dct.keys():
        # dfs_dct[year] = pd.merge(ab_dct["a"], ab_dct["b"], how = "outer", left_index = True, right_index = True)
        dfs_dct[year] = pd.concat([ab_dct["a"], ab_dct["b"]])        
    else:
        dfs_dct[year] = ab_dct["a"]
    dfs_dct[year] = dfs_dct[year]
    # dfs_dct[year].set_index("index", inplace = True)
    # dfs_dct[year].index.name = "Description"

/tmp/ipykernel_26867/2691747984.py:40: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipykernel_26867/2691747984.py:40: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipykernel_26867/2691747984.py:40: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipykernel_26867/2691747984.py:40: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipykernel_26867/2691747984.py:40: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipykernel_26867/2691747984.py:40: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future versio

2017b failed
2018b failed


/tmp/ipykernel_26867/2691747984.py:40: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipykernel_26867/2691747984.py:40: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



2019b failed
2020b failed
2021b failed


/tmp/ipykernel_26867/2691747984.py:40: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/tmp/ipykernel_26867/2691747984.py:40: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [92]:
stack_dfs = pd.concat([dfs_dct[year] for year in dfs_dct.keys()])#.reset_index().set_index(["index", "Year"])

In [135]:
filename = f"{folder}/StateGovFinances2022a{filetype}"
new_df = pd.read_excel(filename, index_col = 0)

new_df = new_df.dropna(axis = 0)
new_df.rename(index = {index: index.upper() for index in new_df.index}, inplace = True)
new_df.index.name = "index"
new_df = new_df.rename(columns = {x: state_dict[x].strip() for x in new_df.columns}).reset_index()
# new_df["index"] = [str(ix).strip().replace("1", "").upper() for ix in new_df['index']]
new_df["index"] = [str(ix).replace("1", "").upper() for ix in new_df['index']]
df_index_count = new_df["index"].groupby(new_df["index"]).cumcount().add(1).astype(str).str.replace("1", "")
new_df["index"] = (df_index_count + new_df["index"]).values
# new_df = new_df.set_index("index")
new_df = new_df.melt(id_vars = ["index"], var_name = "State", value_name = "Value").rename(columns = {"index": "Description"})#.set_index(["State", "Description"]).sort_index()
new_df["Year"] = datetime.datetime(2022,1,1)
# # new_df["Format"] = "State government amount"
new_df = new_df.set_index(["State", "Year"], inplace = False).reset_index()
new_df = new_df.pivot(index = ["State", "Year"], columns = "Description", values = "Value").reindex(new_df["Description"].unique(), axis = 1)
# new_df = new_df[list(new_df.keys())[-1::-1]]
dfs_dct[2022] = new_df.copy()


In [94]:
panel_dfs_dict = {k:stack_dfs[stack_dfs.index.get_level_values(2)==k].reset_index().set_index(["State","Year"]).sort_index() for k in stack_dfs.index.get_level_values(2).unique()}
panel_dfs_dict['State & local government amount'][["1GENERAL REVENUE","1EXPENDITURE"]]

1GENERAL REVENUE 1EXPENDITURE
State Year                                    
AK    1998-01-01          9662079      7753932
      1999-01-01          7611678      8085378
      2000-01-01          9042205      8627637
      2002-01-01          7208865      9397434
      2005-01-01          9824780     10026883
...                           ...          ...
WY    2017-01-01          8728100     10357605
      2018-01-01          8668777     10129438
      2019-01-01          8771293      9986772
      2020-01-01         10018385     10440183
      2021-01-01          9844705     11386915

[1071 rows x 2 columns]

In [95]:
print(list(stack_dfs.keys()))
stack_dfs[["1GENERAL REVENUE","1EXPENDITURE", '1INDIVIDUAL INCOME', '1CORPORATE INCOME']]

['1PERSONAL INCOME', '1REVENUE', '1GENERAL REVENUE', '1INTERGOVERNMENTAL REVENUE', '1FROM FEDERAL GOVERNMENT', '1FROM STATE GOVERNMENT', '1FROM LOCAL GOVERNMENTS', '1GENERAL REVENUE FROM OWN SOURCES', '1TAXES', '1PROPERTY', '1SALES AND GROSS RECEIPTS', '1GENERAL SALES', '1SELECTIVE SALES', '1MOTOR FUEL', '1ALCOHOLIC BEVERAGE', '1TOBACCO PRODUCTS', '1PUBLIC UTILITIES', '1OTHER SELECTIVE SALES', '1INDIVIDUAL INCOME', '1CORPORATE INCOME', '1MOTOR VEHICLE LICENSE', '1OTHER TAXES', '1CHARGES AND MISCELLANEOUS GENERAL  REVENUE', '1CURRENT CHARGES', '1EDUCATION', '1INSTITUTIONS  OF HIGHER EDUCATION', '1SCHOOL LUNCH SALES', '1HOSPITALS', '1HIGHWAYS', '1AIR TRANSPORTATION', '1PARKING FACILITIES', '1SEA AND INLAND PORT FACILITIES', '1NATURAL RESOURCES', '1PARKS AND RECREATION', '1HOUSING AND COMMUNITY DEVELOPMENT', '1SEWERAGE', '1SOLID WASTE MANAGEMENT', '1OTHER CHARGES', '1MISCELLANEOUS GENERAL REVENUE', '1INTEREST EARNINGS', '1SPECIAL ASSESSMENTS', '1SALE OF PROPERTY', '1OTHER GENERAL REVENUE'

1GENERAL REVENUE  \
State Year       Format                                             
US    1998-01-01 State & local government CV                 0.07   
                 State government amount                863522535   
                 Local government amount                794250164   
                 Local government CV                         0.13   
AL    1998-01-01 State & local government amount         18232720   
...                                                           ...   
WY    2021-01-01 State & local government amount          9844705   
                 State & local government CV                 0.19   
                 State government amount                  6375373   
                 Local government amount                  5318614   
                 Local government CV                         0.41   

                                                 1EXPENDITURE  \
State Year       Format                                         
US    1998-01-01 State & local government CV             0.06   
                 State government amount            929951588   
                 Local government amount            884758684   
                 Local government CV                      0.1   
AL    1998-01-01 State & local government amount     21652215   
...                                                       ...   
WY    2021-01-01 State & local government amount     11386915   
                 State & local government CV             0.12   
                 State government amount              7494943   
                 Local government amount              5386454   
                 Local government CV                     0.26   

                                                 1INDIVIDUAL INCOME  \
State Year       Format                                               
US    1998-01-01 State & local government CV                   0.03   
                 State government amount                  160114849   
                 Local government amount                   15515186   
                 Local government CV                           0.32   
AL    1998-01-01 State & local government amount            1887166   
...                                                             ...   
WY    2021-01-01 State & local government amount                  0   
                 State & local government CV                      0   
                 State government amount                          0   
                 Local government amount                          0   
                 Local government CV                              0   

                                                 1CORPORATE INCOME  
State Year       Format                                             
US    1998-01-01 State & local government CV                     0  
                 State government amount                  31088625  
                 Local government amount                   3322990  
                 Local government CV                             0  
AL    1998-01-01 State & local government amount            243803  
...                                                            ...  
WY    2021-01-01 State & local government amount                 0  
                 State & local government CV                     0  
                 State government amount                         0  
                 Local government amount                         0  
                 Local government CV                             0  

[5126 rows x 4 columns]

In [96]:
before2009 = pd.read_csv("StateGovFinancesTo2008.csv", parse_dates=True)
before2009

,State Code,Year4,Sort_Code,Survey Year,ID,GID_Compatible_ID,Type Code,Name,FY Ending Date,Year of Data,...,Unemp-Federal Advances (Y04),Total Worker Comp Rev,Worker Comp-Own Ctrib (Y10),Worker Comp-Oth Ctrib (Y11),Worker Comp-Int Rev (Y12),Worker Comp-Oth Inv Rv (Y13),Tot Oth In Trust Rev,Oth In Trust-Own Ctrib (Y50),Oth In Trust-Oth Ctrib (Y51),Oth In Trust-Int Rev (Y52)
0,0,1902,14893.0,2,00-2,0,2,US STATE GOVTS,NaN,2,...,0,-11111,-11111,-11111,-11111,0,-11111,0,-11111,-11111
1,0,1913,14354.0,13,00-2,0,2,US STATE GOVTS,NaN,13,...,0,-11111,-11111,-11111,-11111,0,-11111,0,-11111,-11111
2,0,1922,13913.0,22,00-2,0,2,US STATE GOVTS,NaN,22,...,0,-11111,-11111,-11111,-11111,0,-11111,0,-11111,-11111
3,0,1927,13668.0,27,00-2,0,2,US STATE GOVTS,NaN,27,...,0,-11111,-11111,-11111,-11111,0,-11111,0,-11111,-11111
4,0,1932,13423.0,32,00-2,0,2,US STATE GOVTS,NaN,32,...,0,-11111,-11111,-11111,-11111,0,-11111,0,-11111,-11111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3254,51,2004,-5696.0,4,51-2,51000000000000,2,WY STATE GOVT,63004.0,4,...,4054,27800,159644,5331,22469,0,0,0,0,0
3255,51,2005,-6114.0,5,51-2,51000000000000,2,WY STATE GOVT,63005.0,5,...,-18,212352,0,191437,20915,0,0,0,0,0
3256,51,2006,-6532.0,6,51-2,51000000000000,2,WY STATE GOVT,63006.0,6,...,-7,249506,0,221952,27554,0,0,0,0,0
3257,51,2007,-6733.0,7,51-2,51000000000000,2,WY STATE GOVT,63007.0,7,...,-1,286602,0,255623,30979,0,0,0,0,0


In [97]:
stack_dfs.to_csv(f"StateGovFinances{start_year}to2021.csv")

In [136]:
gdpefna = pd.read_csv("EFNAGDPTaxes.csv", parse_dates=True).set_index(["State", "Year"])
pop = gdpefna[gdpefna.index.get_level_values("Year") == 2022]["Population"]
pop.reset_index().set_index("State")
dct2022 = {"Level":dfs_dct[2022].reset_index().set_index(["State"]).apply(lambda x: x.mul(1000) if x.dtype == "float64" else x)}
dct2022["Level"]["DEFICIT"] = dct2022["Level"]["TOTAL EXPENDITURE"].sub(dct2022["Level"]["  GENERAL REVENUE"])
del dct2022["Level"]["Year"]
dct2022["Level"] = dct2022["Level"][[list(dct2022["Level"].keys())[-1]] + list(dct2022["Level"].keys()[:-1])]
dct2022["Per Capita"] = dct2022["Level"].apply(lambda x: x.div(pop))
dct2022["Percent of General Revenue"] = dct2022["Level"].apply(lambda x: x.div(dct2022["Level"]["  GENERAL REVENUE"]))
dct2022["Percent of Expenditure"] = dct2022["Level"].apply(lambda x: x.div(dct2022["Level"]["TOTAL EXPENDITURE"]))

In [137]:
import plotly.graph_objects as go
import pandas as pd
from homebrewedFunctions.functions import *
figs = {}
# Assuming dfs_dct and regions_df are already defined as in your original script
for form, plot_df in dct2022.items():
    plot_df = plot_df[plot_df.index.get_level_values("State") != "US"]

    regions_df = pd.read_csv("USCensusRegions.csv")
    plot_df["Region"] = plot_df.index.get_level_values("State").map(regions_df.set_index("State Abbrev")["Region"])
    plot_df["Division"] = plot_df.index.get_level_values("State").map(regions_df.set_index("State Abbrev")["Division"])

    # Define colors for each region
    colors = {}
    for i, region in enumerate(regions_df["Division"].unique()):
        colors[region] = f'rgba({i*30 % 256}, {i*60 % 256}, {i*90 % 256}, 0.8)'

    # List of variables to plot (excluding 'Region' and 'Division')
    variables = [col for col in plot_df.columns if col not in ['Region', 'Division']]

    initial_var = variables[0]

    fig = go.Figure()

    regions = plot_df["Division"].unique()
    x_data = []
    y_data = []

    for region in regions:
        df_region = plot_df[plot_df["Division"] == region]
        x_data_region = df_region.index.get_level_values("State").tolist()
        y_data_region = df_region[initial_var].tolist()
        x_data.append(x_data_region)
        y_data.append(y_data_region)
        fig.add_trace(go.Bar(
            x=x_data_region,
            y=y_data_region,
            name=region,
            marker_color=colors[region]
        ))

    fig.update_layout(
        barmode='group',
        xaxis_tickangle=-45,
        title_text=f'State Government Finances 2022 - {form}: {initial_var}'
    )

    # Create the dropdown menu
    buttons = []
    for var in variables:
        y_data_var = []
        for i, region in enumerate(regions):
            df_region = plot_df[plot_df["Division"] == region]
            y_data_var.append(df_region[var].tolist())
        buttons.append(dict(
            label=var,
            method='update',
            args=[{'y': y_data_var},
                {'title.text': f'State Government Finances 2022 - {form}: {var}'}]
        ))

    fig.update_layout(
        updatemenus=[dict(
            buttons=buttons,
            direction='down',
            showactive=True,
            x=0.0,
            xanchor='left',
            y=1.06,
            yanchor='top'
        )]
    )
    figs[form] = fig
    # fig.show()


figs = dict_of_figs_to_dropdown_fig(figs,show_fig = False, use_sliders = True)
figs.write_html("outputs/State government finances/StateGovFinances2022.html")

/tmp/ipykernel_26867/2000140130.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26867/2000140130.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_26867/2000140130.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykerne